In [16]:
%matplotlib widget

import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from ProcessResults import *
from DatasetTester import *
from GetLandmarks import GetLandmarks 
#from LandmarkTester_alt import LandmarkTester

from landmark_utils import vis_landmarks, compare_landmarks
from vis_utils import *


import cv2
import numpy as np
import time
import timeit
from pathlib import Path
import csv
import pickle

from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.preprocessing import LabelEncoder

from sklearn import metrics
from sklearn.metrics import mean_squared_error
import xgboost as xgb

In [17]:
MEEI_path = Path('./MEEI_Standard_Set')
pat_info = pd.read_excel(MEEI_path/"pat_info.xlsx", sheet_name='Sheet1')

In [18]:
pat_info = pd.read_pickle('MEEI_pat_info.pkl')

with open('all_pats_processed_data.pickle','rb') as f:
    loaded_pats = pickle.load(f)

In [19]:
pat_info

,Category,Sub-category,#,Side,Gender,Age,Path,HB_scale
0,Flaccid,Complete,1,Left,Male,54,MEEI_Standard_Set/Flaccid/CompleteFlaccid/Comp...,6
1,Flaccid,Complete,2,Right,Female,16,MEEI_Standard_Set/Flaccid/CompleteFlaccid/Comp...,6
2,Flaccid,Complete,3,Right,Male,38,MEEI_Standard_Set/Flaccid/CompleteFlaccid/Comp...,6
3,Flaccid,Complete,4,Left,Female,31,MEEI_Standard_Set/Flaccid/CompleteFlaccid/Comp...,6
4,Flaccid,Complete,5,Left,Female,52,MEEI_Standard_Set/Flaccid/CompleteFlaccid/Comp...,6
5,Flaccid,Mild,1,Left,Male,72,MEEI_Standard_Set/Flaccid/MildFlaccid/MildFlac...,3
6,Flaccid,Mild,2,Right,Female,55,MEEI_Standard_Set/Flaccid/MildFlaccid/MildFlac...,3
7,Flaccid,Mild,3,Left,Male,75,MEEI_Standard_Set/Flaccid/MildFlaccid/MildFlac...,3
8,Flaccid,Moderate,1,Left,Male,51,MEEI_Standard_Set/Flaccid/ModerateFlaccid/Mode...,4
9,Flaccid,Moderate,3,Left,Female,62,MEEI_Standard_Set/Flaccid/ModerateFlaccid/Mode...,4


In [20]:
# x - choose one
all_ids = loaded_pats[0]
all_imgs = loaded_pats[1]
all_lms = loaded_pats[2]
all_features = loaded_pats[3]

# y
all_hb = loaded_pats[4]


In [21]:
# Find NaN and Inf values
bad_indices = np.where(np.isnan(all_features).sum(axis=1) > 0)[0]
bad_indices = np.append(bad_indices, np.where(np.isinf(all_features).sum(axis=1) > 0)[0])
bad_indices.sort()
bad_indices

array([], dtype=int64)

In [22]:
# drop bad indices 
# x
all_ids = np.delete(all_ids, bad_indices, axis=0)
all_imgs = np.delete(all_imgs, bad_indices, axis=0)
all_lms = np.delete(all_lms, bad_indices, axis=0)
all_features = np.delete(all_features, bad_indices, axis=0)

# y
all_hb = np.delete(all_hb, bad_indices, axis=0)

In [23]:
# Split IDs into train and test
random.seed(55)
subject_ids = [i for i in range(51)]
train_ids = np.sort(np.array(random.sample(subject_ids, 42)))
test_ids = np.sort(np.array([i for i in subject_ids if i not in train_ids]))

print('Training IDs:',train_ids)
print('Testing IDs:',test_ids)

Training IDs: [ 0  1  2  3  4  5  6  7  9 11 12 13 14 16 18 19 20 21 22 23 24 25 27 28
 29 30 31 32 34 35 36 38 39 40 41 42 43 45 46 47 49 50]
Testing IDs: [ 8 10 15 17 26 33 37 44 48]


In [24]:
# Next, match the IDs to the indices in the data
train_idx = np.sort(np.where(np.isin(all_ids, train_ids))[0])
test_idx = np.sort(np.where(np.isin(all_ids, test_ids))[0])

x_train, x_test = all_features[train_idx], all_features[test_idx]
y_train, y_test = all_hb[train_idx], all_hb[test_idx]

print('Training x shape:', x_train.shape)
print('Training y shape:', y_train.shape)
print('Testing x shape:', x_test.shape)
print('Testing y shape:', y_test.shape)

Training x shape: (336, 29)
Training y shape: (336,)
Testing x shape: (72, 29)
Testing y shape: (72,)


In [25]:
x_train[0], y_train[0]

(array([  1.645194  ,   2.8099802 ,  12.804038  ,   1.0514685 ,
          0.02872195,   0.04908276,   0.22726855,   2.5692458 ,
          1.0851028 ,   1.0569661 ,   1.3587397 ,   0.7349562 ,
          0.68135345,   0.78855604, -14.76165   ,   1.2982798 ,
          1.564689  ,   1.6300427 ,   0.1822432 ,   0.43270245,
          0.4529679 ,   1.7084181 , -11.120797  , -18.251646  ,
          1.0966494 ,   0.7621016 ,   0.7131144 ,   0.29012558,
          0.12468714], dtype=float32),
 6)

In [26]:
x_test[0], y_test[0]

(array([ 0.69027483,  0.5590365 ,  6.7495003 ,  1.0189527 ,  0.01204815,
         0.00975734,  0.11834896, -0.6755946 ,  1.0276055 ,  1.0973325 ,
         1.031084  ,  0.68819636,  0.6809278 ,  0.6930917 , -7.077023  ,
         1.030116  ,  1.6715001 ,  1.525009  ,  0.17358395,  0.3027905 ,
         0.28514296,  1.4757142 , -6.3162465 , -3.7459626 ,  1.0133995 ,
         0.77896845,  0.7644671 ,  0.31192666,  0.14675286], dtype=float32),
 4)

class sklearn.ensemble.RandomForestClassifier(n_estimators=100, *, criterion='gini', max_depth=None, 
                                              min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, 
                                              max_features='sqrt', max_leaf_nodes=None, min_impurity_decrease=0.0, 
                                              bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, 
                                              warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=None)

In [27]:
#le = LabelEncoder()
#y_train = le.fit_transform(y_train)

In [28]:
rf = RFC(random_state=100, n_estimators=30, max_depth=10)
rf.fit(x_train, y_train)

RandomForestClassifier(max_depth=10, n_estimators=30, random_state=100)

In [29]:
# Use the forest's predict method on the test data
y_pred = np.array(rf.predict(x_test))# Calculate the absolute errors
errors = np.square(y_pred - y_test)# Calc Sqrd Errors and Print out the RMSE (Root Mean Squared Error)
print('RMSE:', round(np.sqrt(np.mean(errors)), 2), 'on HB scale.')
print('Accuracy:', round(metrics.accuracy_score(y_test, y_pred), 3))

RMSE: 1.94 on HB scale.
Accuracy: 0.194


In [30]:
y_pred[20:30], y_test[20:30]

(array([1, 1, 1, 1, 3, 5, 1, 5, 3, 3]), array([2, 2, 2, 2, 5, 5, 5, 5, 5, 5]))